# **Loan-Prediction Case Study**


This a beginner-friendly case study on [loan-prediction dataset](https://www.kaggle.com/ninzaami/loan-predication)

## Major topics discussed:
1. Data Preprocesing
2. Data Visualization
3. Pipelines
4. GridSearch
5. Build Custom Estimators

Base Framework borrowed from : [Sagidur's Kernel](https://www.kaggle.com/sazid28/home-loan-prediction), consider visiting this also.


# **Importing necessary packages and data**

Importing neccessary Packages

In [ ]:
# Packages used
!pip install numpy pandas scikit-learn seaborn yellowbrick category_encoders xgboost

In [ ]:
import warnings                                                     # To ignore any warnings
warnings.filterwarnings("ignore")

import numpy as np                                                  # To perform numerical operations on vectors/arrays.
import pandas as pd                                                 # To store and manupulate dataset
import matplotlib.pyplot as plt                                     # To visualize data
import seaborn as sns                                               # To visualize data
from yellowbrick.target import FeatureCorrelation                   # To plot correlation bar chart
from sklearn.model_selection import train_test_split                # To split data to train and test set
from sklearn.preprocessing import  OrdinalEncoder                   # To encode data
from category_encoders import TargetEncoder                         # To encode data
from sklearn.impute import SimpleImputer                            # To impute data
from sklearn.pipeline import Pipeline                               # To build preprocessing and prediction pipeline
from sklearn.compose import ColumnTransformer                       # To apply different tranformation on different columns parallely
from sklearn.base import BaseEstimator, TransformerMixin            # For creating custom estimator
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score                          # Evalution metric that we will be using
from sklearn.model_selection import GridSearchCV, cross_val_score   # To select a descent model

from string import ascii_uppercase                                  # To get alphabetical labels

np.random.seed(42)                                                  # To have same random values on each run.
%matplotlib inline

**Reading the train data**

In [ ]:
data = pd.read_csv('../input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv')
data.sample(10)

**Reading the test data**

In [ ]:
X = data.drop(['Loan_ID','Loan_Status'], axis=1)
y = data['Loan_Status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42, stratify=y)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# **Understanding the Data**

In this section, we will look at the structure of the train and test datasets. Firstly, we will check the features present in our data and then we will look at their data types.


In [ ]:
X_train.columns

We have 12 independent variables and 1 target variable, i.e. Loan_Status in the train dataset.

We will predict the Loan_Status using the model built using the train data.

### Given below is the description for each variable.

##### **Variable	Description** 

* **Loan_ID**:	Unique Loan ID
* **Gender**:	Male/ Female
* **Married**:	Applicant married (Y/N)
* **Dependents**:	Number of dependents
* **Education**:	Applicant Education (Graduate/Under Graduate)
* **Self_Employed**:	Self employed (Y/N)
* **ApplicantIncome**:	Applicant income
* **CoapplicantIncome**:	Coapplicant income
* **LoanAmount**:	Loan amount in thousands
* **Loan_Amount_Term**:	Term of loan in months
* **Credit_History**:	Credit history meets guidelines
* **Property_Area**:	Urban/ Semi Urban/ Rural
* **Loan_Status**:	Loan approved (Y/N)


In [ ]:
# Print data types for each variable
X_train.dtypes

In [ ]:
categorical_features = ['Gender', 'Married', 'Dependents', 'Credit_History',
                        'Self_Employed' , 'Property_Area', 'Education']
numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
                      'Loan_Amount_Term'] 

# Eye-rolling over DataFrame sample you could obseverve tha 'Credit_History'
# is actually a categorical value.

Let’s look at the shape of the dataset.

### Checking whether there exists null values

In [ ]:
X_train.isna().sum()

So, we have null values in our dataset and will have to impute them. We'll do that based upond decision made during **Univariate Analysis**.

## **Univariate Analysis:**

In this section, we will do univariate analysis. It is the simplest form of analyzing data where we examine each variable individually.

For categorical features we can use frequency table or bar plots which will calculate the number of each category in a particular variable. 

For numerical features, probability density plots can be used to look at the distribution of the variable.

### Visualizing Target Variable

We will first look at the target variable, i.e., **Loan_Status**. 

As it is a categorical variable, let us look at its frequency table, percentage distribution and bar plot.

Frequency table of a variable will give us the count of each category in that variable.


In [ ]:
sns.countplot(y_train)

*So our datasets is **unbalanced** and have a Yes:No ratio as 2:1 (approx)*

### Visualizing categorical features


In [ ]:
fig = plt.figure(figsize=(20, 10))

for i,x in enumerate(categorical_features):
    plt.subplot(3, 3, i+1)
    sns.countplot(X_train[x])

plt.suptitle("Categorical Features", fontsize=20)
plt.show()


It can be inferred from the above bar plots that:
* Majority loan applicants in the dataset are male.
* Around 65% of the applicants in the dataset are married.
* Ratio of aplicants with dependents and without is roughly 7:5
* Around 80% of the applicants hold a graduate degree.
* Only about 15% applicants in the dataset are self employed.
* Among applicants distribution of people living in urban, semi-urban and rural is pretty much similar.
* Around 85% applicants have repaid their debts.


### Visualizing numerical features

In [ ]:
fig = plt.figure(figsize=(20, 10))

for i,x in enumerate(numerical_features):
    plt.subplot(2, 2, i+1)
    sns.distplot(X_train[x], kde_kws={'bw':3})

plt.suptitle("Numerical Features", fontsize=20)
plt.show()

It can be seen that in the distributions of **'ApplicantIncome', 'CoapplicantIncome' and 'LoanAmount'** graph is normal distribution skewed towards right. And in case of **Loan_Amount_Term** we see a high concentration around 350 months.

As algorithms works better if the data is normally distributed, we will either cap this or cut the data into bins(i.e; making numerical to categorical)

Lets recall some of the hypotheses that we generated earlier:

•	Applicants with high income should have more chances of loan approval.

•	Applicants who have repaid their previous debts should have higher chances of loan approval.

•	Loan approval should also depend on the loan amount. If the loan amount is less, chances of loan approval should be high.

•	Lesser the amount to be paid monthly to repay the loan, higher the chances of loan approval.

Lets try to test the above mentioned hypotheses using bivariate analysis

After looking at every variable individually in univariate analysis, we will now explore them again with respect to the target variable.


## **Bivariate analysis with respect to output(Loan_Status)**

### Categorical Features

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4)
fig.set_figheight(12)
fig.set_figwidth(20)

i=0

for j,x in enumerate(categorical_features):
    if j>3:
        i=1; j-=4
    _crs_tab = pd.crosstab(X_train[x].dropna(), y_train)
    _crs_tab.div(_crs_tab.sum(1).astype(float), axis=0).plot(kind="bar",
                                                             stacked=True,
                                                             ax=axes[i,j])

plt.suptitle("Categorical Features", fontsize=20)

* It seems people with credit history as 1 are more likely to get their loans approved.
* For all other features approval distribution amongst various classes is more or less the same. 

### Numerical Features

For the ease of visualization and further in preprocessing to handle outliear we will cut the continous varianble into ordinal bins.

In [ ]:
print(numerical_features)

In [ ]:
#list of binned numerical feature vectors 
binned_nums = []

## ApplicantIncome ##
AIbins = [0, 1000, 2000, 3000, 4000, 6000, 8000, 15000, 20000, 40000, 81000]
binned_nums.append(pd.cut(X_train['ApplicantIncome'], bins=AIbins,
                          labels=list(ascii_uppercase[0:10])))

## CoapplicantIncome ##
CIbins = [0, 500, 1000, 2000, 3000, 4000, 5000, 8000, 12000, 20000, 50000]
binned_nums.append(pd.cut(X_train['CoapplicantIncome'], bins=CIbins,
                          labels=list(ascii_uppercase[0:10])))

## LoanAmount ##
LAbins = [0, 50, 100, 200, 500, 1000]
LAlabels = ['very_small_sum', 'small_sum', 'moderate_sum',
            'large_sum', 'very_large_sum']
binned_nums.append(pd.cut(X_train['LoanAmount'], bins=LAbins, labels=LAlabels))


## Loan_Amount_Term ##
LATbins = [0, 60, 360, 600]
LATlabels = ['short_term', 'moderate_term', 'long_term']
binned_nums.append(pd.cut(X_train['Loan_Amount_Term'], bins=LATbins,
                          labels=LATlabels))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_figheight(10)
fig.set_figwidth(20)

i=0

for j,x in enumerate(binned_nums):
    
    if j>1:
        i=1; j-=2
    _crs_tab = pd.crosstab(x.dropna(), y_train)
    _crs_tab.div(_crs_tab.sum(1).astype(float), axis=0).plot(kind="bar",
                                                             stacked=True,
                                                             ax=axes[i,j])

plt.suptitle("Numerical Features (Binned)", fontsize=20)

#### From the graphs we can conclude:

We don't see any clear relation between Applicant income and loan approval but on ther hand we can clearly see a relation coapplicant income and loan amount which is having a lower income co-applicant reaises odds of geeting loan approved. And surety of approval when Co-Applicant Income is very low.
> This is strange. Let's get value counts of **'Coapplicant_Income'**

In [ ]:
X_train['CoapplicantIncome'].value_counts(sort=True)

>The reason we were observing 100% loan approval at very low coapplicant income was because a significant number of applicant do not have a coapplicant at all.

>That makes sense. What we can do is that, we could make a new feature **'Overall_Income'** which would be addition of **'Applicant Income'** and **'Coapplicant Income'**. 

In [ ]:
Overall_Income = X_train['CoapplicantIncome'] + X_train['ApplicantIncome']

OIbins = [0, 2000, 4000, 8000, 16000, 30000, 60000, 100000]
OI_trans = pd.cut(Overall_Income, bins=OIbins,
                          labels=list(ascii_uppercase[0:7]))
_crs_tab = pd.crosstab(OI_trans.dropna(), y_train)
_crs_tab.div(_crs_tab.sum(1).astype(float), axis=0).plot(kind="bar",
                                                         stacked=True)


In [ ]:
corr_matrix = X_train.corr()
corr_matrix

In [ ]:
plt.figure(figsize=(6,6))
sns.heatmap(corr_matrix, annot=True, vmin=-1)

In [ ]:
_temp_X_train = X_train[numerical_features].dropna()
_temp_y_train = y_train[_temp_X_train.index]

vis_pearson = FeatureCorrelation(labels=numerical_features,
                                 method='mutual_info-classification', sort=True)
vis_pearson.fit(_temp_X_train, _temp_y_train)
vis_pearson.poof()



**Mutual information** is a measure between two (possibly multi-dimensional) random variables and , that quantifies the amount of information obtained about one random variable, through the other random variable.

>mutual_info-classification method returns 0 if no correlation and non-zero otherwise.

So, we can remove feature **LoanAmount** and **ApplicantIncome**. Similar inference could also be from the divided bar blot that we earlier plotted.

# **Pre-Processing (Handling null-values and outliers)**

Based upon the insights we got form univariate and bi-variate anlysis we are going process oour data.

Actions/Steps to be performed:
1. Impute Categorical values based on **most_frequent** (mode) value. Using *SimpleImputer* estimator.
2. Impute Numercal values based on **median**.  Using *SimpleImputer* estimator.
3. Cut numerical values to bins. *We will create a **custom estimator** for this step*.
4. Apply *target_encoder* on nominal features.
5. Apply *OrdinalEncoder* on ordinal features.

In [ ]:
# TO-DO
class Nums2Cats(BaseEstimator, TransformerMixin):
    """
    A custom class used to pre-process loan dataset.

    Methods
    -------
    fit(X, y=None)
        Effectively nothing performed in fitting
    transform(X)
        Transforms the given dataframe in to dataframe such that numerical
        features:
            - 'ApplicantIncome' and 'LoanAmount' are dropped
            - 'CoapplicantIncome' and 'Loan_Amount_Term' are converted into
               ordinal categorical values.
    """
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        """
        This method transforms 'CoapplicantIncome' and 'Loan_Amount_Term' are
        converts into ordinal categorical values.
        And drops 'ApplicantIncome' and 'LoanAmount'.

        Parameters
        ----------
        X : pandas.DataFrame
            The dataframe on which processing is needed to be performed.

        Returns
        -------
        pandas.DataFrame
            a processed DataFrame.
        """
        
        # These steps are exactly the same as the one we did above in
        # bivariate visualization with respect to outcome(y_train). 
        # The only difference is that here we use 
        # numpy.digitize() instead of pandas.cut()
        # Please consider going through the documentation of numpy.digitize()
             
        AI_ix, CI_ix, LA_ix, LAT_ix = 0, 1, 2, 3

        ## CoapplicantIncome ##
        CIbins = [500, 1000, 2000, 3000, 4000, 5000, 8000, 12000, 20000]
        CI_cat = np.digitize( (X[:, CI_ix]), bins=CIbins)    


        ## Loan_Amount_Term ##
        LATbins = [60, 360]
        LAT_cat = np.digitize(X[:, LAT_ix], bins=LATbins)

        return np.c_[CI_cat, LAT_cat]

In [ ]:
mode_imp = SimpleImputer(strategy='most_frequent')
median_imp = SimpleImputer(strategy='median')
num2cats = Nums2Cats()

cat_pipe = Pipeline(steps=[('mode_imp', mode_imp)])
num_pipe = Pipeline(steps=[('median_imp', median_imp), ('num2cats', num2cats)])
col_trans_lv1 = ColumnTransformer([('cat_pipe', cat_pipe, categorical_features),
                                   ('num_pipe', num_pipe, numerical_features)])

In [ ]:
numerical_features

In [ ]:
# Testing lvl 1 transformation
test_lvl1 = col_trans_lv1.fit_transform(X_train)  #TO-DO beautify
print(test_lvl1[:5])

In [ ]:
"""To replicate transformations from Num2Cats, for visualizing the tranformation
being made to the numpy array"""

numerical_features_updated = ['CoapplicantIncome', 'Loan_Amount_Term']

print(':: Indices of features after column-transform-lvl1 ::')
dict(enumerate(categorical_features+numerical_features_updated))

Now that we have imputed numerical and categorical fetures and converted numerical features into ordinal categorical features.

Let's now **target_encode** nominal features and **OrdinalEncode** ordinal features 

After **col_trans_lvl1** transformation we will get a numpy.ndarray.

>From our list of feature ordinal feature will be  ***'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Education', 'Dependents'*** out of this except for ***'Education', 'Dependents'*** are already ordinally encoded (thanks to numpy.digitizer).

> And we have nominal features, ***'Gender', 'Married', 'Credit_History', 'Self_Employed', 'Property_Area'***. We will target_encode these features.

In [ ]:
ordinal_feat_ix = [2, 6]             # 'Dependents', 'Education'

nominal_feat_ix = [0, 1, 3, 4, 5]    # 'Gender', 'Married', 'Credit_History',
                                     # 'Self_Employed', 'Property_Area'

In [ ]:
targ_enc = TargetEncoder()
ord_enc = OrdinalEncoder()

col_trans_lv2 = ColumnTransformer([('targ_enc', TargetEncoder(), nominal_feat_ix),
                                   ('ord_enc', OrdinalEncoder(), ordinal_feat_ix)],
                                  remainder='passthrough')

In [ ]:
pre_process_pipe = Pipeline([('col_trans_lvl1', col_trans_lv1),
                             ('col_trans_lvl2', col_trans_lv2)])

# **Model Building and Predictions**
Approach : 
1. We will firt build a base model(logistic regression) and get a base accuracy from the model.
2. Then we will build several complex models and judge them basing the base model.
3. We will pick the most promising model and then fine tune its hyper-parameter.

### Base Model (Logistic Regression)

In [ ]:
y_train = y_train.replace(['Y', 'N'], [1, 0])   # replacing Y=1 and N=0

X_train_processed = pre_process_pipe.fit_transform(X_train, y_train)

In [ ]:
base_model = LogisticRegression()
base_model.fit(X_train_processed, y_train)

Let’s predict the Loan_Status for validation set and calculate its accuracy.

In [ ]:
train_prediction = base_model.predict(X_train_processed)

base_accuracy_train = round(accuracy_score(y_train, train_prediction)*100, 2)
base_accuracy_cv = round(np.mean(cross_val_score(base_model, X_train_processed,
                                                 y_train, scoring='accuracy',
                                                 cv=10, n_jobs=-1))*100, 2)
print(':: Base Accuracy :: \nTrain :: {0} \nCV :: {1}'.format(base_accuracy_train,
                                                              base_accuracy_cv))

**We will take these as our base accuracy now, that is our next model must give atleast this much accuracy.**

### Model Selection

We will build the following models using our processed dataset and check how it perform with base model.

* Logistic Regression
* Decision Tree
* Random Forest
* XGBoost
* Multi-layer Perceptron
* KNN
* Naive-Bayes Classifier
* SVM

In [ ]:
dtc = DecisionTreeClassifier(random_state=42)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
mlpc = MLPClassifier(random_state=42)
knn = KNeighborsClassifier(n_jobs=-1)
gnb = GaussianNB()
svc = SVC(random_state=42)
xgb = XGBClassifier(n_jobs=-1)

classifiers = [dtc, rfc, mlpc, knn, gnb, svc, xgb]
train_accuracies = dict()
cv_accuracies = dict()

In [ ]:
for clf in classifiers:
    clf.fit(X_train_processed, y_train)
    y_train_pred = clf.predict(X_train_processed)
    train_accuracies[type(clf).__name__] = round(accuracy_score(y_train_pred,
                                                                y_train)*100, 2)
    cv_accuracies[type(clf).__name__] = round(np.mean(cross_val_score(clf,
                                        X_train_processed, y_train,
                                        scoring='accuracy',
                                        cv=20, n_jobs=-1))*100, 2)

In [ ]:
pd.DataFrame([train_accuracies, cv_accuracies], index=['Train', 'CV'])

### Model tuning (GaussainNB)

From the above DataFrame we can see that **Gaussian Naive-Bayes** gives the best performance out of all models on default parameters.
Also a 11.42 % accuracy from the base model.

> Decision Tree and XGBClassifier are having a better accuracy on training set but quite low on CV set, which indicates models are overfittin training data.

We will use GridSearchCV for hyper parameter tuning

In [ ]:
# GaussianNB have very few hyperparameter to tune.
params = { 'var_smoothing': [1e-30, 1e-20, 1e-10, 1] }

grdsrch_clf = GridSearchCV(gnb, params, cv=20)
grdsrch_clf.fit(X_train_processed, y_train)

In [ ]:
print("Best Score CV :: {0} \n"
      "Best Estimator :: {1}".format(grdsrch_clf.best_score_,
                                     grdsrch_clf.best_estimator_))

# **Prediction**
As, now we have our best estimator. Lets evaluate our model on test set.

In [ ]:
# First we have to pre-process the data
X_test_processed = pre_process_pipe.transform(X_test)

# As we trained on outcomes as [0,1] output values will be also in [0,1]
y_test = y_test.replace(['Y', 'N'], [1, 0])

y_pred_test = grdsrch_clf.predict(X_test_processed)
test_accuracy = round(accuracy_score(y_test, y_pred_test), 2)
print('Accuracy of our model :: ', test_accuracy)

Thats a descent accuracy score from our model.

# Addional Notes
If you don't completely understand the topics discussed or you feel like having a bit more details, it is completely OK.

Here are a few quick reads worth going through after the case study.
1. [Seaborn Visualiztions](https://medium.com/swlh/introduction-to-data-visualization-with-seaborn-6232b70e9b30)
2. [Choosing Metrics](https://medium.com/usf-msds/choosing-the-right-metric-for-machine-learning-models-part-1-a99d7d7414e4)
3. [Sklearn-Pipelines](https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf)
4. [Custom Estimators](http://danielhnyk.cz/creating-your-own-estimator-scikit-learn/)
5. [KNN](https://machinelearningmastery.com/k-nearest-neighbors-for-machine-learning/)
6. [Decision Trees](https://medium.com/@rishabhjain_22692/decision-trees-it-begins-here-93ff54ef134)
7. [Naive-Bayes](https://www.machinelearningplus.com/predictive-modeling/how-naive-bayes-algorithm-works-with-example-and-full-code/)
8. [SVM](https://medium.com/data-science-simplified/svm-support-vector-machine-beginners-ml-series-part-1-the-theory-part-7c618ef8ff9b)
9. [XGBoost](https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7)
10. [MLP](https://medium.com/@AI_with_Kain/understanding-of-multilayer-perceptron-mlp-8f179c4a135f)

I hope this helps you.

# Thank You